In [ ]:
import pandas as pd
import sqlalchemy as sa
from urllib.parse import quote
import datetime
import seaborn as sns
from datetime import datetime, timedelta
import datetime

In [ ]:
cur_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print("current times:", cur_time)

In [ ]:
df = pd.read_csv("data/dataset.csv", encoding="cp874", )

In [ ]:
df.info()

Func show label

In [ ]:
def annotate_vertical_plot(ax):
  for p in ax.patches:
    value = p.get_height()
    position = (p.get_x() + p.get_width() / 2, p.get_height())
    ax.annotate(f"{value: .0f}", position, ha ="center", va="bottom")

ref data dic : https://apps-doe.moph.go.th/boe/software/file/R506_v_4_10.pdf

ประเภทเชื้อ

In [ ]:
ax = sns.countplot(data=df, x="DISEASE", )
annotate_vertical_plot(ax)

เพศ 1 ชาย 2 หญิง

In [ ]:
ax = sns.countplot(data=df , x="SEX")
annotate_vertical_plot(ax)

สถานะ  1 โสด 2 คู่ 3 หย่าร้าง 4 หม้าย 5 ไม่ทราบ

In [ ]:
ax = sns.countplot(data=df , x="MARIETAL")
annotate_vertical_plot(ax)

สัญชาติ 1 ไทย 2 จีน/ฮ่องกง/ไต้หวัน 3 พม่า 4 มาเลเซีย 5 กัมพูชา 6 ลาว 7 เวียดนาม 8 อื่น

In [ ]:
# def change_race(txt) :
#    race_dict = {
#         "ไทย": 1,
#         "จีน": 2,
#         "ฮ๋องกง": 2,
#         "ไต้หวัน": 2,
#         "พม่า": 3,
#         "มาเลเซีย": 4,
#         "กัมพูชา": 5,
#         "ลาว": 6,
#         "เวียดนาม": 7,
#         "อื่นๆ": 8
#     }
#    if txt in race_dict:
#       return race_dict[txt]
#    else:
#       return "ไม่พบข้อมูล"

In [ ]:
##print(change_race("ไทย"))

In [ ]:
df.loc[df['RACE'] == 'ไทย', 'RACE'] = '1'
df.loc[df['RACE'] == 'จีน', 'RACE'] = '2'
df.loc[df['RACE'] == 'ฮ๋องกง', 'RACE'] = '2'
df.loc[df['RACE'] == 'ไต้หวัน', 'RACE'] = '2'
df.loc[df['RACE'] == 'พม่า', 'RACE'] = '3'
df.loc[df['RACE'] == 'มาเลเซีย', 'RACE'] = '4'
df.loc[df['RACE'] == 'กัมพูชา', 'RACE'] = '5'
df.loc[df['RACE'] == 'ลาว', 'RACE'] = '6'
df.loc[df['RACE'] == 'เวียดนาม', 'RACE'] = '7'
df.loc[df['RACE'] == 'อื่นๆ', 'RACE'] = '8'


In [ ]:
ax = sns.countplot(data=df , x="RACE")
annotate_vertical_plot(ax)

In [ ]:
# ax = sns.countplot(data=df , x="DATESICK")
# annotate_vertical_plot(ax)

ผู้ป่วยรายอำเภอ

In [ ]:
df['NADDRCODE'] = df.ADDRCODE.apply(lambda x: x[:4])
df['NADDRCODE']

In [ ]:
patient_grouped = df.groupby([ 'NADDRCODE']).size().reset_index()
print(patient_grouped)

In [ ]:
ax = sns.countplot(data=df , x="NADDRCODE")
annotate_vertical_plot(ax)

In [ ]:
new_df = df[~df["NADDRCODE"].str.contains('96')]
new_df

เอาเฉพาะผู้ป่วยที่อยุ่นราธิวาส

In [ ]:
df = df[df["NADDRCODE"].str.contains('96')]
df

In [ ]:
# Strip any leading/trailing whitespace from the date strings
df['DATESICK'] = df['DATESICK'].str.strip()

# Convert 'DATESICK' column to datetime
df['DATESICK'] = pd.to_datetime(df['DATESICK'], format='%d/%m/%Y')

print(df)
# print(df.dtypes)

In [ ]:
# from datetime import datetime
def calculate_week_number(str_date):
    # Start of the custom week 1
    # str_date = datetime.strptime(str_date, format).date()
    start_of_week_1 = pd.Timestamp(year=str_date.year, month=1, day=7)
    # print(start_of_week_1)
    # if str_date < start_of_week_1:
    #     start_date = datetime.date(str_date.year, 1, 7)
    if str_date < start_of_week_1:
        return 52  # For dates before the start of week 1
    return ((str_date - start_of_week_1).days // 7) + 1

In [ ]:
# Sample data with date as string
# data = {
#     'date': ['02/01/2024', '07/01/2024', '06/07/2024'],
#     'cases': [5, 10, 3]
# }
# da = pd.DataFrame(data)

# da['ndate'] = da['date'].str.strip()

# # Convert 'DATESICK' column to datetime
# da['ndate'] = pd.to_datetime(da['ndate'], format='%d/%m/%Y')
# da.info()
# da['week_number'] = da['ndate'].apply(calculate_week_number)
# print(da)

In [ ]:
# Apply the function to get the week number
#df['week_number'] = df['DATESICK'].apply(calculate_week_number)

# Group by week number and sum cases
# weekly_cases = da.groupby('week_number')['ndate'].count().reset_index()

# print(weekly_cases)

หาผู้ป่วย รายอำเภอ

In [ ]:
# Apply the function to get the week number
df['week_number'] = df['DATESICK'].apply(calculate_week_number)

# Group by week number and sum cases
#weekly_cases = df.groupby('week_number','NADDRCODE')['DATESICK'].count().reset_index()
case_on_week_by_district = df.groupby(['week_number', 'NADDRCODE']).size().reset_index(name='DATESICK')
print(case_on_week_by_district)

In [ ]:
def find_week_number(date):
    """
    Calculate the week number of a given date, where week 1 starts on January 7.

    Args:
    date (datetime.date): The date to calculate the week number for.

    Returns:
    int: The week number of the date.
    """
    # Define the starting date of week 1
    start_date = datetime.date(date.year, 1, 7)
    
    # If the given date is before January 7 of the same year, adjust the start date to the previous year's January 7
    if date < start_date:
        start_date = datetime.date(date.year - 1, 1, 7)
    
    # Calculate the difference in days
    days_diff = (date - start_date).days
    
    # Calculate the week number
    week_number = (days_diff // 7) + 1
    
    return week_number

def get_start_of_week(week_number, year):
    """
    Calculate the start date of a given week number, where week 1 starts on January 7.

    Args:
    week_number (int): The week number.
    year (int): The year.

    Returns:
    datetime.date: The start date of the week.
    """
    start_date = datetime.date(year, 1, 7)
    start_of_week = start_date + datetime.timedelta(weeks=week_number - 1)
    return start_of_week

def create_weeks_dataframe(start_date_str, end_date_str):
    """
    Create a DataFrame with weeks starting from January 7, along with their start and end dates.

    Args:
    start_date_str (str): The start date string for the DataFrame in 'YYYY-MM-DD' format.
    end_date_str (str): The end date string for the DataFrame in 'YYYY-MM-DD' format.

    Returns:
    pd.DataFrame: A DataFrame with week numbers and their start and end dates.
    """
    start_date = datetime.datetime.strptime(start_date_str, '%Y-%m-%d').date()
    end_date = datetime.datetime.strptime(end_date_str, '%Y-%m-%d').date()
    
    weeks = []
    current_date = start_date
    
    while current_date <= end_date:
        week_number = find_week_number(current_date)
        start_of_week = get_start_of_week(week_number, current_date.year)
        end_of_week = start_of_week + datetime.timedelta(days=6)
        weeks.append({
            'week_number': week_number,
            'start_date': start_of_week,
            'end_date': end_of_week
        })
        current_date = end_of_week + datetime.timedelta(days=1)
    
    weeks_df = pd.DataFrame(weeks).drop_duplicates(subset=['week_number'])
    return weeks_df

In [ ]:
# Example usage:
start_date = '2024-01-07'
end_date = '2025-01-04'
weeks_df = create_weeks_dataframe(start_date, end_date)
weeks_df

In [ ]:
weeks_df

In [ ]:
ndf = case_on_week_by_district.merge(weeks_df, on='week_number', how='left')
ndf

In [ ]:
new_df = pd.merge(case_on_week_by_district, weeks_df , on=['week_number'])
new_df = new_df[['week_number','NADDRCODE', 'start_date', 'end_date', 'DATESICK']]
new_df

In [ ]:
new_df.rename(columns={'DATESICK': "total_case", 'NADDRCODE' : "district_code"}, inplace=True)
new_df

In [ ]:
sns.lineplot(data=new_df, x='week_number',y='total_case')

In [ ]:
new_df.info()

Insert data to db 

In [ ]:
DIALECT = "mysql"
SQL_DRIVER = "pymysql"
USERNAME = "user"
PASSWORD = "user"
HOST = "localhost"
PORT = 6001
DBNAME = "r506"

conn_str = DIALECT + "+" + SQL_DRIVER + "://" + USERNAME + ":" +quote(PASSWORD) + "@" + HOST + ":" +str(PORT) + "/" + DBNAME

In [ ]:
with sa.create_engine(conn_str).connect() as con:
  new_df.to_sql("cases",con,index=None, if_exists='replace')